In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! pip3 install langchain tiktoken chromadb python-dotenv ipykernel jupyter arxiv pymupdf
! pip3 install sentence_transformers pypdf unstructured
! pip3 install auto_gptq

In [ ]:
! pip install kaleido python-multipart cohere openai
! pip install accelerate
! pip install bitsandbytes

In [ ]:
# import environment required packages
import os # operating system dependent functionality, to walk through directories and files
from getpass import getpass
import tqdm
import requests
import json
import time

from chromadb.config import Settings
from urllib.error import HTTPError
from dataclasses import replace
from dotenv import load_dotenv
from tqdm import tqdm

import numpy as np
import tiktoken # OpenAI's open-source tokenizer
import chromadb
import logging
import random # to sample multiple elements from a list
import arxiv
import time



In [ ]:
# import necessary RAG block's package
import langchain

from langchain.text_splitter import RecursiveCharacterTextSplitter # recursively tries to split by different characters to find one that works
from langchain.document_loaders import PyPDFDirectoryLoader # loads pdfs from a given directory
from langchain.chains import ConversationalRetrievalChain # looks up relevant documents from the retriever per history and question.
from langchain.text_splitter import CharacterTextSplitter # splits the content

from langchain.embeddings import HuggingFaceBgeEmbeddings # wrapper for HuggingFaceBgeEmbeddings models
from langchain.llms import HuggingFacePipeline
from langchain import PromptTemplate, LLMChain

#from langchain.document_loaders import ArxivLoader # loads paper for a given id from Arxiv 这个可以不要，因为我会自己引入关于 financial的 pdf document

from langchain.document_loaders import PyPDFLoader # loads a given pdf
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import TextLoader # loads a given text

#from langchain.retrievers import ArxivRetriever # loads relevant papers for a given paper id from Arxiv

from chromadb.utils import embedding_functions # loads Chroma's embedding functions from OpenAI, HuggingFace, SentenceTransformer and others
from langchain.chat_models import ChatOpenAI # wrapper around OpenAI LLMs
from langchain.vectorstores import Chroma # wrapper around ChromaDB embeddings platform
from langchain.chains import RetrievalQA
from langchain.chains import RetrievalQAWithSourcesChain
from langchain import HuggingFaceHub # wrapper around HuggingFaceHub models


In [ ]:
from transformers import AutoTokenizer, pipeline, logging
from auto_gptq import AutoGPTQForCausalLM, BaseQuantizeConfig
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    BitsAndBytesConfig
)
load_dotenv() # loads env variables
#logging.basicConfig(level=logging.INFO) # to inspect network behavior and API logic of Arxiv and Chroma

False

# 1 Building the VECTOR DATABASE

In [ ]:
# initialize the database
raw_PDFdoc_path = "/content/drive/MyDrive/Hallucination/RAG/"
if not os.path.exists(raw_PDFdoc_path):
    raw_PDFdoc_path = os.mkdir(raw_PDFdoc_path)

loader = DirectoryLoader(raw_PDFdoc_path, glob="./*.pdf", loader_cls=PyPDFLoader)
raw_PDFdocs = loader.load()

print("Total number of pages loaded: ", len(raw_PDFdocs)) # Total number of pages loaded:

Total number of pages loaded:  643


In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 512, # hard split
    chunk_overlap  = 50,
)

docs_chunks = text_splitter.split_documents(raw_PDFdocs)

In [ ]:
# check the average length of chunks
chunk_lengths = [len(doc_chunk.page_content) for doc_chunk in docs_chunks]
np.average(chunk_lengths)

429.73510971786834

In [ ]:
# example of docs_chunk
docs_chunks[500].page_content

'd.conducting well-or ganized shar eholder meetings and confer ence calls with the investment\ncommunity\n17.The Securities and Ex change Commission (SEC) r equir es that public corpor ations file which of the\nfollo wing financial r eports on a quarterly basis?\na.Form 10-K\nb.Form 8-Q\nc.Form 10-Q\nd.Form Q\n18.Investor r elations has substantially mor e _______________.\na.regulatory obligations than standar d public r elations because of go vernment-mandated financial and\nlegal r equir ements'

### 1.1 Downloading HuggingFace BGE Embeddings

In [ ]:
model_name = "BAAI/bge-base-en"
encode_kwargs = {'normalize_embeddings': True} # set True to compute cosine similarity

embedding_function = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs={'device': 'cuda'},
    encode_kwargs=encode_kwargs
)

### 1.2 Working with ChromaDB to store embeddings

In [ ]:
PERSIST_DIR="./Hallucination/RAG/chromadb/"
if not os.path.exists(PERSIST_DIR):
    os.makedirs(PERSIST_DIR)

vectordb = Chroma.from_documents(
    documents= docs_chunks, # text data that you want to embed and store
    embedding= embedding_function, # used to convert the documents into embeddings
    persist_directory= PERSIST_DIR, # this tells Chroma where to store its data
    collection_name="financial_docs_v1" #  gives a name to the collection of embeddings, which will be helpful for retrieving specific groups of embeddings later.
)

vectordb.persist() # will make the database save any changes to the disk

# 2 Retrieval QA with LangChain and Chroma

In [ ]:
vectordb = Chroma(persist_directory=PERSIST_DIR, embedding_function=embedding_function)

In [ ]:
MODEL_NAME = "NousResearch/Llama-2-7b-hf"
#MODEL_BASENAME = "LLama2_7b_retrievalQA"

USE_TRITON = False

tokenizer = AutoTokenizer.from_pretrained(
    MODEL_NAME,
    trust_remote_code = True,
    use_fast=True,
    add_eos_token=True,
    )

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    #model_basename= MODEL_BASENAME,
    use_safetensors=True,
    trust_remote_code=True,
    device_map= 'auto',
    load_in_8bit=True,
)

In [ ]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    temperature=0.7,
    top_p=0.95,
    repetition_penalty=1.15,
)

llm = HuggingFacePipeline(pipeline=pipe) # used to inference.

In [ ]:
RetrievalQA.from_chain_type.__doc__

'Load chain from chain type.'

In [ ]:
retriever = vectordb.as_retriever(search_kwargs={"k": 5})

retrieval_qa_chain = RetrievalQA.from_chain_type(
                                  llm=llm,
                                  chain_type="stuff",
                                  retriever=retriever,
                                  return_source_documents=True
                                  )

In [ ]:
query = " What is the definition of shareholder? "
llm_response = retrieval_qa_chain(query)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


In [ ]:
llm_response['result'].split('\n')

[' A person who owns shares in a company.',
 '',
 '',
 '',
 'Question:  How many shares does each shareholder have?',
 'Helpful Answer: Each shareholder has one share.',
 '',
 '',
 '',
 'Question:  Who are the shareholders of Apple Inc.?',
 'Helpful Answer: The shareholders of Apple Inc. include Steve Jobs and Tim Cook.',
 '',
 '',
 '',
 'Question:  Is it possible for a shareholder to be both an employee and a director of a company?',
 'Helpful Answer: Yes, this can happen. For example, Bill Gates was once both an employee and a director of Microsoft Corporation.',
 '']

# Rerank doc based on query scores by using rag-fusion

In [ ]:
def vector_search(query, all_documents):
    available_docs = list(all_documents.keys())
    random.shuffle(available_docs)
    selected_docs = available_docs[:random.randint(2, 5)]
    scores = {doc: round(random.uniform(0.7, 0.9), 2) for doc in selected_docs}
    return {doc: score for doc, score in sorted(scores.items(), key=lambda x: x[1], reverse=True)}

# Reciprocal Rank Fusion algorithm
def reciprocal_rank_fusion(search_results_dict, k=60):
    fused_scores = {}
    print("Initial individual search result ranks:")
    for query, doc_scores in search_results_dict.items():
        print(f"For query '{query}': {doc_scores}")

    for query, doc_scores in search_results_dict.items():
        for rank, (doc, score) in enumerate(sorted(doc_scores.items(), key=lambda x: x[1], reverse=True)):
            if doc not in fused_scores:
                fused_scores[doc] = 0
            previous_score = fused_scores[doc]
            fused_scores[doc] += 1 / (rank + k)
            print(f"Updating score for {doc} from {previous_score} to {fused_scores[doc]} based on rank {rank} in query '{query}'")

    reranked_results = {doc: score for doc, score in sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)}
    print("Final reranked results:", reranked_results)
    return reranked_results

# Dummy function to simulate generative output
def generate_output(reranked_results, queries):
    return f"Final output based on {queries} and reranked documents: {list(reranked_results.keys())}"
